In [18]:
%tensorflow_version 1.x
import tensorflow.keras as K
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [19]:
def preprocess_data(X, Y):
    X = K.applications.resnet50.preprocess_input(X)
    Y = K.utils.to_categorical(Y, 10)

    return X, Y

In [20]:
# Divide the data in Train and Test Datasets
(x_train, y_train),(x_test, y_test) = K.datasets.cifar10.load_data()

x_train, y_train = preprocess_data(x_train, y_train)
x_test, y_test = preprocess_data(x_test, y_test)

# The original size of pictures is 32 x 32, we need to upscale to 244 x 224 to 
# fit on Resnet50 
target_size = (224, 224)

model = K.applications.ResNet50(include_top=False,
                                weights='imagenet',
                                input_shape=(224, 224, 3))

In [21]:
for layer in model.layers[:-32]:
    layer.trainable=False

In [22]:
model_1= K.Sequential()
model_1.add(K.layers.Lambda(lambda image: tf.image.resize(image, target_size))) 
model_1.add(model)
model_1.add(K.layers.GlobalAveragePooling2D()) 
model_1.add(K.layers.Flatten())
model_1.add(K.layers.Dense(64, activation='relu', kernel_initializer='he_normal'))
model_1.add(K.layers.Dense(10,activation='softmax', kernel_initializer='he_normal'))

checkpoint = K.callbacks.ModelCheckpoint(filepath='cifar10.h5',
                                          monitor='val_acc',
                                          mode='max',
                                          verbose=1,
                                          save_best_only=True)

model_1.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['acc'])

model_1.fit(x_train, y_train,
            validation_data=(x_test, y_test),
            batch_size=32,
            epochs=10,
            verbose=1,
            callbacks=[checkpoint])

model_1.save('cifar10.h5')

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
49984/50000 [============================>.] - ETA: 0s - loss: 0.5048 - acc: 0.8341
Epoch 00001: val_acc improved from -inf to 0.78970, saving model to cifar10.h5
50000/50000 [==============================] - 166s 3ms/sample - loss: 0.5048 - acc: 0.8341 - val_loss: 1.0650 - val_acc: 0.7897
Epoch 2/10
49984/50000 [============================>.] - ETA: 0s - loss: 0.2348 - acc: 0.9198
Epoch 00002: val_acc improved from 0.78970 to 0.85290, saving model to cifar10.h5
50000/50000 [==============================] - 153s 3ms/sample - loss: 0.2349 - acc: 0.9198 - val_loss: 0.7617 - val_acc: 0.8529
Epoch 3/10
49984/50000 [============================>.] - ETA: 0s - loss: 0.1345 - acc: 0.9545
Epoch 00003: val_acc improved from 0.85290 to 0.86530, saving model to cifar10.h5
50000/50000 [==============================] - 153s 3ms/sample - loss: 0.1345 - acc: 0.9545 - val_loss: 0.6495 - val_acc: 0.8653
Epoch 4/10
49984/50000 [===========

In [7]:
model_1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              multiple                  0         
_________________________________________________________________
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl multiple                  0         
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  131136    
_________________________________________________________________
dense_1 (Dense)              multiple                  650       
Total params: 23,719,498
Trainable params: 15,107,786
Non-trainable params: 8,611,712
____________________________________

In [9]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________